In [157]:
rm(list=ls())  #start fresh

In [158]:
install.packages("openxlsx") #install the package needed to read the file

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [159]:
#access the library to read the file
library(openxlsx)


In [160]:
#read the file
HumDevIndx = read.xlsx("https://github.com/Fundamentals-Josh/Week_6_Exercise_2/raw/refs/heads/main/HDR25_Statistical_Annex_HDI_Table.xlsx")

In [161]:
#show the head for examination (10 rows)
head(HumDevIndx,10)

,X1,Table.1..Human.Development.Index.and.its.components,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,NA,NA,NA,SDG3,NA,SDG4.3,NA,SDG4.4,NA,SDG8.5,NA,NA,NA,NA
2,NA,NA,Human Development Index (HDI),Life expectancy at birth,NA,Expected years of schooling,NA,Mean years of schooling,NA,Gross national income (GNI) per capita,NA,GNI per capita rank minus HDI rank,NA,HDI rank
3,HDI rank,Country,Value,(years),NA,(years),NA,(years),NA,(2021 PPP $),NA,NA,NA,NA
4,NA,NA,2023,2023,NA,2023,a,2023,a,2023,NA,2023,b,2022
5,NA,Very high human development,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1,Iceland,0.97199999999999998,82.691000000000003,NA,18.850589750000001,c,13.908926279999999,d,69116.937359999996,NA,12,NA,3
7,2,Norway,0.97,83.308000000000007,NA,18.792850489999999,c,13.117962179999999,e,112710.0211,f,0,NA,1
8,2,Switzerland,0.97,83.953999999999994,NA,16.667530060000001,NA,13.94912109,e,81948.901769999997,f,5,NA,2
9,4,Denmark,0.96199999999999997,81.933000000000007,NA,18.704010010000001,c,13.027320599999999,e,76007.856690000001,f,4,NA,4


In [162]:
#delete unecessary columns and show execution
HumDevIndx$X5 <- NULL
HumDevIndx$X7 <- NULL
HumDevIndx$X9 <- NULL
HumDevIndx$X11 <- NULL
HumDevIndx$X13 <- NULL

head(HumDevIndx,3)

,X1,Table.1..Human.Development.Index.and.its.components,X3,X4,X6,X8,X10,X12,X14
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,NA,NA,NA,SDG3,SDG4.3,SDG4.4,SDG8.5,NA,NA
2,NA,NA,Human Development Index (HDI),Life expectancy at birth,Expected years of schooling,Mean years of schooling,Gross national income (GNI) per capita,GNI per capita rank minus HDI rank,HDI rank
3,HDI rank,Country,Value,(years),(years),(years),(2021 PPP $),NA,NA


In [163]:
# Read the file again, this time without assuming headers initially.
# This ensures column names are generic (V1, V2, etc.) before we explicitly set them.
HumDevIndx = read.xlsx("https://github.com/Fundamentals-Josh/Week_6_Exercise_2/raw/refs/heads/main/HDR25_Statistical_Annex_HDI_Table.xlsx", colNames = FALSE)

# Delete unnecessary columns first using numeric indices.
# delete columns 5, 7, 9, 11, 13.
HumDevIndx <- HumDevIndx[, -c(5, 7, 9, 11, 13)]

# Manually define the correct header names based on the original row 2 and the columns kept.
# This prevents issues where HumDevIndx[2, ] might contain data values or NA's.
correct_headers <- c(
  "2023 HDI rank",
  "Country",
  "Human Development Index (HDI)",
  "Life expectancy at birth",
  "Expected years of schooling",
  "Mean years of schooling",
  "Gross national income (GNI) per capita",
  "GNI per capita rank minus HDI rank",
  "HDI rank"
)

# Assign these as the new column names
colnames(HumDevIndx) <- correct_headers

# Trim whitespace from all column names to ensure exact matches for future operations.
names(HumDevIndx) <- trimws(names(HumDevIndx))

# Now, remove the metadata rows. Original rows 1 (table title), 3 (category title),
# and 4 (a duplicate 'HDI rank' header), 5 (metadata year), and 6 (region/group) are no longer needed as data rows.
# Since we used original row 2 as headers, we now remove current rows 1, 2, 3, 4, 5, 6.
HumDevIndx <- HumDevIndx[-c(1:6), ]

# Reset the row names to be a continuous sequence starting from 1.
rownames(HumDevIndx) <- NULL

# Ensure dplyr is loaded for next stpes
library(dplyr)

# Convert specified columns to integer
HumDevIndx <- HumDevIndx %>%
  mutate(
    `2023 HDI rank` = as.integer(`2023 HDI rank`),
    `GNI per capita rank minus HDI rank` = as.integer(`GNI per capita rank minus HDI rank`)
  )

# Ensure 'Country' is character type
HumDevIndx$Country <- as.character(HumDevIndx$Country)

# Identify columns to convert to numeric (double to preserve decimal places)
numeric_cols <- c(
  "Human Development Index (HDI)",
  "Life expectancy at birth",
  "Expected years of schooling",
  "Mean years of schooling",
  "Gross national income (GNI) per capita"
)

# Apply as.double to these columns. Adding a gsub for 'Human Development Index (HDI)'
# to remove any non-numeric characters (like hidden spaces) that might prevent proper conversion.
HumDevIndx <- HumDevIndx %>%
  mutate(
    `Human Development Index (HDI)` = as.double(gsub("[^0-9.]", "", `Human Development Index (HDI)`)),
    across(all_of(setdiff(numeric_cols, "Human Development Index (HDI)")), as.double)
  )

# Round specified numeric columns to two decimal places for display purposes
HumDevIndx <- HumDevIndx %>%
  mutate(
    `Human Development Index (HDI)` = round(`Human Development Index (HDI)`, 2),
    `Life expectancy at birth` = round(`Life expectancy at birth`, 2),
    `Expected years of schooling` = round(`Expected years of schooling`, 2),
    `Mean years of schooling` = round(`Mean years of schooling`, 2),
    `Gross national income (GNI) per capita` = round(`Gross national income (GNI) per capita`, 2)
  )

# Finally, rename the "HDI rank" column to "2022 HDI rank" as requested, and convert it to integer.
HumDevIndx <- HumDevIndx %>%
  rename("2022 HDI rank" = "HDI rank") %>%
  mutate(`2022 HDI rank` = as.integer(`2022 HDI rank`))


#show head and string for verification
head(HumDevIndx)
str(HumDevIndx)

Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `GNI per capita rank minus HDI rank = as.integer(`GNI per capita
  rank minus HDI rank`)`.
Caused by warning:
! NAs introduced by coercion”
Warning message:
“There were 3 warnings in `mutate()`.
The first warning was:
ℹ In argument: `Human Development Index (HDI) = as.double(gsub("[^0-9.]", "",
  `Human Development Index (HDI)`))`.
Caused by warning:
! NAs introduced by coercion
ℹ Run `dplyr::last_dplyr_warnings()` to see the 2 remaining warnings.”
Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `2022 HDI rank = as.integer(`2022 HDI rank`)`.
Caused by warning:
! NAs introduced by coercion”


,2023 HDI rank,Country,Human Development Index (HDI),Life expectancy at birth,Expected years of schooling,Mean years of schooling,Gross national income (GNI) per capita,GNI per capita rank minus HDI rank,2022 HDI rank
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>
1,1,Iceland,0.97,82.69,18.85,13.91,69116.94,12,3
2,2,Norway,0.97,83.31,18.79,13.12,112710.02,0,1
3,2,Switzerland,0.97,83.95,16.67,13.95,81948.90,5,2
4,4,Denmark,0.96,81.93,18.70,13.03,76007.86,4,4
5,5,Germany,0.96,81.38,17.31,14.30,64053.22,13,6
6,5,Sweden,0.96,83.26,18.99,12.74,66102.09,10,4


'data.frame':	262 obs. of  9 variables:
 $ 2023 HDI rank                         : int  1 2 2 4 5 5 7 8 8 10 ...
 $ Country                               : chr  "Iceland" "Norway" "Switzerland" "Denmark" ...
 $ Human Development Index (HDI)         : num  0.97 0.97 0.97 0.96 0.96 0.96 0.96 0.96 0.96 0.95 ...
 $ Life expectancy at birth              : num  82.7 83.3 84 81.9 81.4 ...
 $ Expected years of schooling           : num  18.9 18.8 16.7 18.7 17.3 ...
 $ Mean years of schooling               : num  13.9 13.1 13.9 13 14.3 ...
 $ Gross national income (GNI) per capita: num  69117 112710 81949 76008 64053 ...
 $ GNI per capita rank minus HDI rank    : int  12 0 5 4 13 10 14 4 6 9 ...
 $ 2022 HDI rank                         : int  3 1 2 4 6 4 8 9 7 13 ...


In [164]:
library(dplyr) # Ensure dplyr is loaded

# Convert specified columns to integer
HumDevIndx <- HumDevIndx %>%
  mutate(
    `2023 HDI rank` = as.integer(`2023 HDI rank`),
    `GNI per capita rank minus HDI rank` = as.integer(`GNI per capita rank minus HDI rank`),
    `2022 HDI rank` = as.integer(`2022 HDI rank`)
  )

# Ensure 'Country' is character type
HumDevIndx$Country <- as.character(HumDevIndx$Country)

# Convert other numerical columns to numeric to preserve decimal places
# This includes 'Human Development Index (HDI)', 'Life expectancy at birth',
# 'Expected years of schooling', 'Mean years of schooling',
# and 'Gross national income (GNI) per capita' (for dollars)

# Identify columns to convert to numeric
numeric_cols <- c(
  "Human Development Index (HDI)",
  "Life expectancy at birth",
  "Expected years of schooling",
  "Mean years of schooling",
  "Gross national income (GNI) per capita"
)

# Apply as.double to these columns. Adding a gsub for 'Human Development Index (HDI)'
# to remove any non-numeric characters (like hidden spaces) that might prevent proper conversion.
HumDevIndx <- HumDevIndx %>%
  mutate(
    `Human Development Index (HDI)` = as.double(gsub("[^0-9.]", "", `Human Development Index (HDI)`)),
    across(all_of(setdiff(numeric_cols, "Human Development Index (HDI)")), as.double)
  )

# To display with 2 decimal places for relevant columns, you'd typically format for output
# rather than changing the underlying data type. For example:
# HumDevIndx$`Human Development Index (HDI)` <- round(HumDevIndx$`Human Development Index (HDI)`, 2)
# The `head()` output might truncate decimal display, but `str()` will confirm the underlying type.

head(HumDevIndx)
str(HumDevIndx)

,2023 HDI rank,Country,Human Development Index (HDI),Life expectancy at birth,Expected years of schooling,Mean years of schooling,Gross national income (GNI) per capita,GNI per capita rank minus HDI rank,2022 HDI rank
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>
1,1,Iceland,0.97,82.69,18.85,13.91,69116.94,12,3
2,2,Norway,0.97,83.31,18.79,13.12,112710.02,0,1
3,2,Switzerland,0.97,83.95,16.67,13.95,81948.90,5,2
4,4,Denmark,0.96,81.93,18.70,13.03,76007.86,4,4
5,5,Germany,0.96,81.38,17.31,14.30,64053.22,13,6
6,5,Sweden,0.96,83.26,18.99,12.74,66102.09,10,4


'data.frame':	262 obs. of  9 variables:
 $ 2023 HDI rank                         : int  1 2 2 4 5 5 7 8 8 10 ...
 $ Country                               : chr  "Iceland" "Norway" "Switzerland" "Denmark" ...
 $ Human Development Index (HDI)         : num  0.97 0.97 0.97 0.96 0.96 0.96 0.96 0.96 0.96 0.95 ...
 $ Life expectancy at birth              : num  82.7 83.3 84 81.9 81.4 ...
 $ Expected years of schooling           : num  18.9 18.8 16.7 18.7 17.3 ...
 $ Mean years of schooling               : num  13.9 13.1 13.9 13 14.3 ...
 $ Gross national income (GNI) per capita: num  69117 112710 81949 76008 64053 ...
 $ GNI per capita rank minus HDI rank    : int  12 0 5 4 13 10 14 4 6 9 ...
 $ 2022 HDI rank                         : int  3 1 2 4 6 4 8 9 7 13 ...


In [165]:
# Remove rows where '2023 HDI rank' is NA (blank)

HumDevIndx <- HumDevIndx[!is.na(HumDevIndx$`2023 HDI rank`), ]

# Reset row names to a continuous sequence starting from 1
rownames(HumDevIndx) <- NULL

# Display the head of the data frame to verify
head(HumDevIndx)

,2023 HDI rank,Country,Human Development Index (HDI),Life expectancy at birth,Expected years of schooling,Mean years of schooling,Gross national income (GNI) per capita,GNI per capita rank minus HDI rank,2022 HDI rank
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>
1,1,Iceland,0.97,82.69,18.85,13.91,69116.94,12,3
2,2,Norway,0.97,83.31,18.79,13.12,112710.02,0,1
3,2,Switzerland,0.97,83.95,16.67,13.95,81948.90,5,2
4,4,Denmark,0.96,81.93,18.70,13.03,76007.86,4,4
5,5,Germany,0.96,81.38,17.31,14.30,64053.22,13,6
6,5,Sweden,0.96,83.26,18.99,12.74,66102.09,10,4


In [166]:
#show the tail to ensure that the changes made it through the entire data frame
tail(HumDevIndx,25)

,2023 HDI rank,Country,Human Development Index (HDI),Life expectancy at birth,Expected years of schooling,Mean years of schooling,Gross national income (GNI) per capita,GNI per capita rank minus HDI rank,2022 HDI rank
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>
169,169,Senegal,0.53,68.68,9.14,2.93,4202.03,-12,169
170,170,Gambia,0.52,65.86,8.98,4.67,2812.02,1,170
171,171,Congo (Democratic Republic of the),0.52,61.90,10.91,7.38,1431.05,17,172
172,172,Malawi,0.52,67.35,9.90,5.24,1633.69,12,173
173,173,Benin,0.52,60.77,10.45,3.19,3805.53,-13,174
174,174,Guinea-Bissau,0.51,64.08,10.64,3.67,2403.06,2,175
175,175,Djibouti,0.51,65.99,6.21,3.95,6367.61,-36,176
176,176,Sudan,0.51,66.33,8.62,3.96,2810.31,-4,171
177,177,Liberia,0.51,62.16,10.47,6.17,1537.86,9,177


In [167]:
#calculating means:
#Life expectancy at birth (rounded to 2 decimal places)
#Expected years of schooling (rounded to 2 decimal places)
#Mean years of schooling(rounded to 2 decimal places)
#Gross national income (GNI) per capita (rounded to 2 decimal places)


round(mean(HumDevIndx$`Life expectancy at birth`, na.rm = TRUE), 2)
round(mean(HumDevIndx$`Expected years of schooling`, na.rm = TRUE), 2)
round(mean(HumDevIndx$'Mean years of schooling', na.rm = TRUE), 2)
round(mean(HumDevIndx$'Gross national income (GNI) per capita', na.rm = TRUE), 0)

[1] 73.11

[1] 13.58

[1] 9.17

[1] 24621